# Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import pickle

# Chargement des données

In [ ]:
# Chargement des données
def load_cifar10_batch(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    data = dict[b'data']
    labels = dict[b'labels']
    return data, labels

def load_cifar10_data():
    X_train = []
    y_train = []

    for i in range(1, 6):
        data_batch, labels_batch = load_cifar10_batch(f'DatasetCIFAR10/data_batch_{i}')
        X_train.append(data_batch)
        y_train.extend(labels_batch)

    X_train = np.concatenate(X_train)
    y_train = np.array(y_train)

    X_test, y_test = load_cifar10_batch('DatasetCIFAR10/test_batch')
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = load_cifar10_data()

# Traitement des données

In [ ]:
# Reshape et normalisation des données
X_train = X_train.reshape(-1, 3, 32, 32) / 255.0
X_test = X_test.reshape(-1, 3, 32, 32) / 255.0

# Conversion en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# DataLoaders

In [ ]:
# Création de l'ensemble de validation
train_size = int(0.8 * len(X_train_tensor))
val_size = len(X_train_tensor) - train_size

train_dataset, val_dataset = random_split(
    TensorDataset(X_train_tensor, y_train_tensor), 
    [train_size, val_size]
)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# GPU

In [ ]:
# Vérification de l'utilisation du GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Réseau de Neurones

In [ ]:
# Définition du CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 8 * 8 *4, 256)
        self.fc2 = nn.Linear(256, 10)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
model = SimpleCNN().to(device)

# Optimiseur

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Fonction de perte

In [ ]:
criterion = nn.CrossEntropyLoss()

# Entraînement

In [ ]:
epochs = 20
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    val_accuracy = correct / total
    val_accuracies.append(val_accuracy)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Visualisation de l'entraînement

In [ ]:
# Graphiques de l'évolution de l'entraînement
plt.figure()
plt.plot(range(1, epochs+1), train_losses, label="Train Loss")
plt.plot(range(1, epochs+1), val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.title("Loss Evolution")
plt.show()

# Visualisation Précision Validation

In [ ]:
plt.figure()
plt.plot(range(1, epochs+1), val_accuracies, label="Validation Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Accuracy Evolution")
plt.show()

# Evaluation du réseau de neurones

In [ ]:
# Évaluation finale sur l'ensemble de test
model.eval()
y_true = []
y_pred = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Déplacement des données sur le GPU
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())  # Déplacement sur le CPU pour l'évaluation
        y_pred.extend(predicted.cpu().numpy())

test_accuracy = accuracy_score(y_true, y_pred)
print(f"Test Accuracy: {test_accuracy:.4f}")
